In [ ]:
import sys 
sys.path.append('../../')

from math import asin 

from core.passive_learning import *
from core.auto_labeling import *
from utils.logging_utils import * 
from datasets import dataset_factory  
from datasets.dataset_utils import * 
from utils.counts import *  
from utils.common_utils import * 
from utils.vis_utils import *
import copy 
import random 
from calibration.calibration_utils import * 

from core.run_lib import *

# configuration
conf_dir = '../../configs/arxiv-configs/Cifar10-Vit/'

pas_learn_conf_file = '{}/passive_learning_cifar10_vit_pytorch.yaml'.format(conf_dir)

conf = load_yaml_config(pas_learn_conf_file)

logger = get_logger('../../temp/logs/pl.log','PL',level=logging.DEBUG)

conf['data_conf']['data_path'] = '../../data/'
conf['data_conf']['val_fraction'] = 0.5
train_conf = conf['training_conf']

train_conf['optimizer']='sgd'
train_conf['learning_rate']= 0.04
train_conf['momentum']=0.9
train_conf['weight_decay']=5e-4 

train_conf['train_err_tol'] = -1

train_conf['batch_size']= 64 #len(train_set)
train_conf['shuffle']= True
train_conf['max_epochs'] = 10

train_conf['log_val_err']=True 
conf['auto_lbl_conf']['auto_label_err_threshold'] = 0.1
conf['auto_lbl_conf']['C_1'] = 0.01

In [ ]:

results = []
i = 1
n = 10000

conf["random_seed"]=i

conf['training_conf']['seed_train_size'] = 2500

conf['training_conf']['train_set_frac'] = n/(50000 *(1-conf['data_conf']['val_fraction']))
set_seed(conf['random_seed'])

# get data
ds = dataset_factory.load_dataset(conf)
ds.build_dataset()
train_set,val_set_std = randomly_split_dataset(ds,1- conf['data_conf']['val_fraction'])
# this val_set is the fixed val_set of big size. the algo will be given a subsample of it.

auto_lbl_conf = conf['auto_lbl_conf']

val_set_subset = val_set_std.get_random_fraction(auto_lbl_conf['val_frac_for_auto_lbl'])

test_set = ds.get_test_datasets()
logger.info('Loaded dataset {}'.format(conf['data_conf']['dataset']))
logger.info('Unlabeled pool size: {} Validation Set Size:{}'.format(len(train_set),len(val_set_subset)))

human_labeling_helper = HumanLabelingHelper(train_set.X,train_set.Y)

pl = PassiveLearning(conf, train_set, human_labeling_helper, ds_val=val_set_subset,ds_test=test_set,logger=logger)
out = pl.run()
pl.get_counts(train_set,test_set)
